# 다양한 청킹 및 임베딩 기법 비교

### **학습 목표:**
- 다양한 청킹(Chinking) 전략의 차이점을 이해하고, RAG 성능에 미치는 영향을 분석합니다.
  - SimpleChunker: 고정 길이 기준

  - SentenceChunker: 문장 단위로 잘라서 길이 조정

  - RecursiveCharacterTextSplitter: LangChain 기본

- 임베딩 모델을 활용하여 분할된 텍스트를 벡터화하고, VectorStore에 저장하는 과정을 실습합니다.

In [6]:
# # 기본 langchain 패키지들 설치
# !pip install langchain langchain-community langchain-experimental

# # 추가로 필요한 패키지들 설치
# !pip install pypdf pypdf2 pymupdf pdfplumber faiss-cpu sentence-transformers langchain_huggingface ipywidgets

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

# 한국어 특화 SentencePiece 기반 임베딩 모델
model_name = "jhgan/ko-sroberta-multitask"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [9]:
# 임베딩 테스트
test_text = "안녕하세요, 한국어 임베딩 테스트입니다."
result = embeddings.embed_query(test_text)
print(f"✅ 임베딩 벡터 차원: {len(result)}")
print("✅ 한국어 임베딩 모델이 정상 작동합니다!")

✅ 임베딩 벡터 차원: 768
✅ 한국어 임베딩 모델이 정상 작동합니다!


In [10]:
# SentencePiece 사용 확인
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sroberta-multitask")
print(tokenizer.backend_tokenizer)

Tokenizer(version="1.0", truncation=TruncationParams(direction=Right, max_length=128, strategy=LongestFirst, stride=0), padding=PaddingParams(strategy=BatchLongest, direction=Right, pad_to_multiple_of=None, pad_id=1, pad_type_id=0, pad_token="[PAD]"), added_tokens=[{"id":0, "content":"[CLS]", "single_word":False, "lstrip":False, "rstrip":False, ...}, {"id":1, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, ...}, {"id":2, "content":"[SEP]", "single_word":False, "lstrip":False, "rstrip":False, ...}, {"id":3, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, ...}, {"id":4, "content":"[MASK]", "single_word":False, "lstrip":False, "rstrip":False, ...}], normalizer=BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False), pre_tokenizer=BertPreTokenizer(), post_processor=TemplateProcessing(single=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0)], pair=[Speci

## 샘플 문서 로드
비교를 위해 의미 있는 내용이 충분히 담긴 문서를 로드합니다.  
여기서는 **한국지능정보사회진흥원**의 AI국가전략 보고서 중 "전국민 AI 일상화 실행 계획"을 사용하겠습니다.
 - https://www.nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx=99952&bcIdx=27378&parentSeq=27378

In [11]:
import requests

# PDF 다운로드 URL
pdf_url = "https://www.nia.or.kr/common/board/Download.do?bcIdx=27378&cbIdx=99952&fileNo=1"
pdf_filename = "National_AI_Plan.pdf"

# 파일 저장
with open(pdf_filename, "wb") as f:
    f.write(requests.get(pdf_url).content)

print("PDF 파일 저장 완료:", pdf_filename)

PDF 파일 저장 완료: National_AI_Plan.pdf


In [12]:
# import sys
# print("주피터 Python 경로:", sys.executable)

In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("National_AI_Plan.pdf")
documents = loader.load()

## 문서를 잘 읽고 있는지 페이지를 달리하면서, 확인해보세요
print(documents[15].page_content[:500])

-6-
 Ⅲ. 국내 현황 진단◈전 세계적 디지털 모범국가로 나아가기 위한 정부의 전략 추진과민간의 선제적 혁신으로 전국민 AI일상화초석 마련○(정부)디지털 심화 시대에 대응한 신속한 전략･
정책추진을 통해 AI기술･
산업 경쟁력을 강화하고 민간 혁신을 뒷받침  * 대한민국 디지털 전략(‘22.9) → AI일상화·산업 고도화계획(’23.1)→초거대 AI경쟁력강화방안(‘23.4)-｢
AI법제정비단(’23.8~)｣
를 통해 AI확산에 따른 사회적 이슈를 정비하고,AI거버넌스 정립과 공동 번영을 위한 글로벌 협력 본격화○(민간)독자적 초거대 AI개발･
출시 본격화,중소･
스타트업 중심의다양한 응용서비스출시 등 산업 생태계 조성시작네이버, ‘클로바’LG, ‘엑사원’카카오, KoGPT SKT, ‘에이닷’KT, ‘믿음’
-의료 등 다양한 도메인에서의중소·스타트업 역시 초거대 AI기반전문서비스를 출시,체감되는 AI활용·확산을 주도
* 의료 AI 기업 ‘루닛’이 美정부의 암정복 프로젝트 ‘캔서 문샷


## 청킹(Chunking) 전략 비교
동일한 문서를 세가지 다른 방식으로 분활하여 각 전략의 특징을 살펴보겠습니다.  
1. **CharacterTextSplitter**  
   가장 기본적인 분할기. 지정된 문자를 기준으로 텍스트를 나눕니다. 문장의 의미 구조를 고려하지 않아 문맥이 깨질 수 있습니다.

2. **RecursiveCharacterTextSplitter**  
   LangChain의 기본 분할기. `["\n\n", "\n", " ", ""]` 등 다양한 구분자를 재귀적으로 시도하여 문맥을 최대한 유지하려고 노력합니다.

3. **SemanticChunker**  
   임베딩 벡터 간의 유사도를 기반으로 의미적으로 완전한 덩어리를 찾습니다. 고정된 크기가 아닌, 의미의 완결성을 기준으로 분할하는 가장 진보된 방식입니다.

### CharacterTextSplitter
- 가장 기본적인 청킹(splitter) 방식
- 지정된 구분자(separator)로 텍스트를 분할 후,
- chunk_size(최대 길이)와 chunk_overlap(중첩 길이) 만큼 슬라이딩 윈도우 방식으로 청크 생성
- 구분자(문단, 줄바꿈 등)만을 기준으로 동작 (문장, 단어 등 의미 단위는 고려하지 않음)
- 매우 빠르고 직관적이지만, 문장 중간 등에서 잘릴 수 있음

#### 파라미터
- `separator` : 청킹 기준이 되는 구분자 (ex: "\n", "\n\n", " ") (예) "\n\n" : 문단 단위
- `chunk_size` : 한 청크의 최대 길이 (문자 단위)
- `chunk_overlap` : 청크 간 겹치는 길이 (문자 단위)

In [14]:
# 1. CharacterTextSplitter
print("\n--- 1. CharacterTextSplitter 실행 중... ---")
char_splitter = CharacterTextSplitter(separator="", chunk_size=500, chunk_overlap=100)
char_chunks = char_splitter.split_documents(documents)
for i, chunk in enumerate(char_chunks[:5]):
    print(f"청크 예시:[{i}]")
    print(f"CharacterTextSplitter로 분할된 청크 수: {len(char_chunks)}")
    print("Content:")
    print(chunk.page_content)
    print("-" * 50)


--- 1. CharacterTextSplitter 실행 중... ---
청크 예시:[0]
CharacterTextSplitter로 분할된 청크 수: 88
Content:
전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동
--------------------------------------------------
청크 예시:[1]
CharacterTextSplitter로 분할된 청크 수: 88
Content:
순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ. 추진과제 10  1. AI로 국민 일상을 풍요롭게 하겠습니다10  2. AI 내재화로 산업･일터를 혁신하겠습니다15  3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20  4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------------------------------------
청크 예시:[2]
CharacterTextSplitter로 분할된 청크 수: 88
Content:
요  약
--------------------------------------------------
청크 예시:[3]
CharacterTextSplitter로 분할된 청크 수: 88
Content:
-i-
전국민 AI 일상화 실행계획(안) (요약)Ⅰ추진 배경□새정부 출범이후 ‘뉴욕구상(’22.9)’,‘대한민국 디지털 전략(’22.9)’을통해 국민과 함께 세계의 모범이 되는 디지털 강국실현 추진   * ‘파리 이니셔티브(’23.6)’를 통해 새로운 디지털 질서･규범 정립 본격화○디지털 핵심인 AI경쟁력 강화를 위해 ‘AI일상화 및 산업 고도화계획(’23.1)‘,’초거대AI경쟁력 강화 방안(‘23.4)’등 정책적 노력 지속   * ‘캐나다 AI 석학과의 대화(’22.9)’를 통해 우리나라 AI가 나아가야 할 방향 모색□국민과 AI혜택을 공유하고 AI를 가장 잘 

청크 예시 [3] 뒷부분과 [4] 앞부분을 보면 overlap이 되는 걸 확인할 수 있습니다.  
이렇게 파라미터 조정으로 쉽게 문서를 분할하고 overlap도 할 수 있습니다.  

### To Do
각각의 파라미터를 조정하면서 결과를 확인해보세요.  
`separator` 예시 : "\n\n", "\n", " ", ".", ",", "" 등  
`overlap`을 제대로 확인하고 싶다면 구분자를 "" (문자 단위)로 하는 것이 좋습니다. 

In [15]:
# # 예제 1: 문단 단위로 분할 (추천)
# print("\n--- 1. 문단 단위 분할 ---")
# char_splitter = CharacterTextSplitter(
#     separator="\n\n",  # 빈 줄로 문단 구분
#     chunk_size=1000,   # 1000자 단위
#     chunk_overlap=100, # 100자 겹침
# )
# char_chunks = char_splitter.split_documents(documents)
# for i, chunk in enumerate(char_chunks[:3]):
#     print(f"청크 {i+1}: {len(chunk.page_content)}자")
#     print(f"내용: {chunk.page_content[:100]}...")
#     print("-" * 50)

# # 예제 2: 문장 단위로 분할
# print("\n--- 2. 문장 단위 분할 ---")
# sentence_splitter = CharacterTextSplitter(
#     separator=". ",    # 마침표와 공백으로 문장 구분
#     chunk_size=500,    # 500자 단위
#     chunk_overlap=50,  # 50자 겹침
# )
# sentence_chunks = sentence_splitter.split_documents(documents)
# for i, chunk in enumerate(sentence_chunks[:3]):
#     print(f"청크 {i+1}: {len(chunk.page_content)}자")
#     print(f"내용: {chunk.page_content[:100]}...")
#     print("-" * 50)

# # 예제 3: 줄 단위로 분할
# print("\n--- 3. 줄 단위 분할 ---")
# line_splitter = CharacterTextSplitter(
#     separator="\n",    # 줄바꿈으로 구분
#     chunk_size=800,    # 800자 단위
#     chunk_overlap=80,  # 80자 겹침
# )
# line_chunks = line_splitter.split_documents(documents)
# for i, chunk in enumerate(line_chunks[:3]):
#     print(f"청크 {i+1}: {len(chunk.page_content)}자")
#     print(f"내용: {chunk.page_content[:100]}...")
#     print("-" * 50)

# # 예제 4: 한국어 특화 문장 분할
# print("\n--- 4. 한국어 문장 분할 ---")
# korean_splitter = CharacterTextSplitter(
#     separator="다. ",  # 한국어 문장 끝 패턴
#     chunk_size=600,    # 600자 단위
#     chunk_overlap=60,  # 60자 겹침
# )
# korean_chunks = korean_splitter.split_documents(documents)
# for i, chunk in enumerate(korean_chunks[:3]):
#     print(f"청크 {i+1}: {len(chunk.page_content)}자")
#     print(f"내용: {chunk.page_content[:100]}...")
#     print("-" * 50)

# # 청크 통계 출력
# print(f"\n=== 분할 결과 통계 ===")
# print(f"문단 분할: {len(char_chunks)}개 청크")
# print(f"문장 분할: {len(sentence_chunks)}개 청크")
# print(f"줄 분할: {len(line_chunks)}개 청크")
# print(f"한국어 분할: {len(korean_chunks)}개 청크")

In [16]:
# 1. CharacterTextSplitter - 파라미터 변경 실습
print("\n--- 1. CharacterTextSplitter 실행 중... ---")
char_splitter = CharacterTextSplitter(
    # separator="\n\n",  # 빈 줄로 문단 구분
    # separator="\n",    # 줄바꿈으로 구분
    separator=". ",  # 마침표와 공백으로 문장 구분
    chunk_size=500,  # 청크 사이즈를 바꿔보세요.
    chunk_overlap=50,  # 겹치고 싶은 문자 수를 입력해보세요.
)
char_chunks = char_splitter.split_documents(documents)
for i, chunk in enumerate(char_chunks[:5]):
    print(f"청크 예시:[{i}]")
    print(f"CharacterTextSplitter로 분할된 청크 수: {len(char_chunks)}")
    print("Content:")
    print(chunk.page_content)
    print("-" * 50)

Created a chunk of size 581, which is longer than the specified 500



--- 1. CharacterTextSplitter 실행 중... ---
청크 예시:[0]
CharacterTextSplitter로 분할된 청크 수: 53
Content:
전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동
--------------------------------------------------
청크 예시:[1]
CharacterTextSplitter로 분할된 청크 수: 53
Content:
순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ. 추진과제 10  1. AI로 국민 일상을 풍요롭게 하겠습니다10  2. AI 내재화로 산업･일터를 혁신하겠습니다15  3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20  4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------------------------------------
청크 예시:[2]
CharacterTextSplitter로 분할된 청크 수: 53
Content:
요  약
--------------------------------------------------
청크 예시:[3]
CharacterTextSplitter로 분할된 청크 수: 53
Content:
-i-
전국민 AI 일상화 실행계획(안) (요약)Ⅰ추진 배경□새정부 출범이후 ‘뉴욕구상(’22.9)’,‘대한민국 디지털 전략(’22.9)’을통해 국민과 함께 세계의 모범이 되는 디지털 강국실현 추진   * ‘파리 이니셔티브(’23.6)’를 통해 새로운 디지털 질서･규범 정립 본격화○디지털 핵심인 AI경쟁력 강화를 위해 ‘AI일상화 및 산업 고도화계획(’23.1)‘,’초거대AI경쟁력 강화 방안(‘23.4)’등 정책적 노력 지속   * ‘캐나다 AI 석학과의 대화(’22.9)’를 통해 우리나라 AI가 나아가야 할 방향 모색□국민과 AI혜택을 공유하고 AI를 가장 잘 

### RecursiveCharacterTextSplitter
- CharacterTextSplitter의 업그레이드 버전
- 여러 구분자(문단, 줄바꿈, 문장, 띄어쓰기 등)를 우선순위에 따라 순차적으로 적용
    → 최대한 자연스럽게(문장/문단 단위) 자르려고 시도
- 만약 지정한 구분자에서 잘라서 `chunk_size`를 만족하지 못하면,
    → 더 작은 구분자로 재귀적으로 내려가서 청크를 만듦
- 자연스러운 문맥 유지가 필요할 때 유용

#### 파라미터
- `chunk_size	` : 한 청크의 최대 길이 (문자 단위)  
- `chunk_overlap` : 청크 간 겹치는 길이 (문자 단위)  
- `separators` : 구분자 우선순위 리스트 (ex: ["\n\n", "\n", " ", ""]), 기본값: ["\n\n", "\n", " ", ""]  
- `keep_separator` : 구분자를 청크에 포함할지 여부 (True/False)  

In [17]:
# 2. RecursiveCharacterTextSplitter
print("\n--- 2. RecursiveCharacterTextSplitter 실행 중... ---")
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, chunk_overlap=0, separators=["\n\n", "\n", " ", ""], keep_separator=False
)
recursive_chunks = recursive_splitter.split_documents(documents)
for i, chunk in enumerate(recursive_chunks[:5]):
    print(f"청크 예시:[{i}]")
    print(f"CharacterTextSplitter로 분할된 청크 수: {len(recursive_chunks)}")
    print("Content:")
    print(chunk.page_content)
    print("-" * 50)


--- 2. RecursiveCharacterTextSplitter 실행 중... ---
청크 예시:[0]
CharacterTextSplitter로 분할된 청크 수: 120
Content:
전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동
--------------------------------------------------
청크 예시:[1]
CharacterTextSplitter로 분할된 청크 수: 120
Content:
순 서 Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ. 추진과제 10 1. AI로 국민 일상을 풍요롭게 하겠습니다10 2. AI 내재화로 산업･일터를 혁신하겠습니다15 3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20 4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------------------------------------
청크 예시:[2]
CharacterTextSplitter로 분할된 청크 수: 120
Content:
요 약
--------------------------------------------------
청크 예시:[3]
CharacterTextSplitter로 분할된 청크 수: 120
Content:
-i-
--------------------------------------------------
청크 예시:[4]
CharacterTextSplitter로 분할된 청크 수: 120
Content:
전국민 AI 일상화 실행계획(안) (요약)Ⅰ추진 배경□새정부 출범이후 ‘뉴욕구상(’22.9)’,‘대한민국 디지털 전략(’22.9)’을통해 국민과 함께 세계의 모범이 되는 디지털 강국실현 추진 * ‘파리 이니셔티브(’23.6)’를 통해 새로운 디지털 질서･규범 정립 본격화○디지털 핵심인 AI경쟁력 강화를 위해 ‘AI일상화 및 산업 고도화계획(’23.1)‘,’초거대A

### To Do
파라미터에 따라 어떻게 결과가 달라지는지 확인해보세요.

In [19]:
# 방법 1: 주피터 출력 제한 해제
import sys

import pandas as pd
from IPython.display import display

# 출력 제한 해제
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# sys 출력 제한 해제 (선택적)
# sys.stdout = sys.__stdout__

print("🔧 출력 제한 해제 완료!")


# 방법 2: 스마트한 샘플링 출력
def smart_display_chunks(chunks, name, max_display=3, show_stats=True):
    """청크를 스마트하게 출력하는 함수"""
    print(f"\n{'='*60}")
    print(f"📊 {name}")
    print(f"{'='*60}")

    if show_stats:
        total_chunks = len(chunks)
        avg_length = sum(len(chunk.page_content) for chunk in chunks) / total_chunks
        min_length = min(len(chunk.page_content) for chunk in chunks)
        max_length = max(len(chunk.page_content) for chunk in chunks)

        print(f"📈 통계:")
        print(f"  • 총 청크 수: {total_chunks}")
        print(f"  • 평균 길이: {avg_length:.1f}자")
        print(f"  • 최소 길이: {min_length}자")
        print(f"  • 최대 길이: {max_length}자")
        print()

    # 샘플 청크 출력 (첫 번째, 중간, 마지막)
    sample_indices = []
    if len(chunks) > 0:
        sample_indices.append(0)  # 첫 번째
    if len(chunks) > 2:
        sample_indices.append(len(chunks) // 2)  # 중간
    if len(chunks) > 1:
        sample_indices.append(len(chunks) - 1)  # 마지막

    # 중복 제거
    sample_indices = list(set(sample_indices))[:max_display]

    for idx in sample_indices:
        chunk = chunks[idx]
        print(f"📄 청크 {idx+1}/{len(chunks)} ({len(chunk.page_content)}자):")
        print(f"   {chunk.page_content[:200]}...")
        print()


# 방법 3: 전체 출력 강제 표시
def force_full_output():
    """주피터 출력 제한 완전 해제"""
    from IPython.core.interactiveshell import InteractiveShell

    InteractiveShell.ast_node_interactivity = "all"

    # 출력 라인 제한 해제
    import IPython.core.oinspect

    IPython.core.oinspect.page = lambda x: print(x)


# 방법 4: 파일로 저장하여 확인
def save_chunks_to_file(chunks, filename, name):
    """청크 결과를 파일로 저장"""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"=== {name} 결과 ===\n\n")
        f.write(f"총 청크 수: {len(chunks)}\n\n")

        for i, chunk in enumerate(chunks):
            f.write(f"--- 청크 {i+1} ({len(chunk.page_content)}자) ---\n")
            f.write(chunk.page_content)
            f.write("\n\n" + "=" * 80 + "\n\n")

    print(f"💾 결과가 {filename}에 저장되었습니다.")


# 실제 사용 예제
print("🚀 RecursiveCharacterTextSplitter 파라미터별 비교")

# 예제 1: 기본 설정
recursive_splitter_1 = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", ". ", " ", ""], keep_separator=False
)
chunks_1 = recursive_splitter_1.split_documents(documents)
smart_display_chunks(chunks_1, "기본 설정 (chunk_size=1000)")

# 예제 2: 작은 청크
recursive_splitter_2 = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""], keep_separator=False
)
chunks_2 = recursive_splitter_2.split_documents(documents)
smart_display_chunks(chunks_2, "작은 청크 (chunk_size=300)")

# 예제 3: 한국어 특화 + 구분자 유지
recursive_splitter_3 = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=80,
    separators=["\n\n", "\n", "다. ", "요. ", "는다. ", ". ", ", ", " ", ""],
    keep_separator=True,  # 구분자 유지
)
chunks_3 = recursive_splitter_3.split_documents(documents)
smart_display_chunks(chunks_3, "한국어 특화 + 구분자 유지")

# 예제 4: 큰 겹침 (문맥 보존)
recursive_splitter_4 = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=300, separators=["\n\n", "\n", " ", ""], keep_separator=False  # 큰 겹침
)
chunks_4 = recursive_splitter_4.split_documents(documents)
smart_display_chunks(chunks_4, "큰 겹침 (overlap=300)")

# 최종 비교 테이블
print(f"\n{'='*80}")
print("📊 전체 결과 비교")
print(f"{'='*80}")
comparison_data = [
    ["기본 설정", len(chunks_1), 1000, 100, "기본 구분자"],
    ["작은 청크", len(chunks_2), 300, 50, "기본 구분자"],
    ["한국어 특화", len(chunks_3), 600, 80, "한국어 구분자 + keep_separator"],
    ["큰 겹침", len(chunks_4), 800, 300, "기본 구분자"],
]

print(f"{'설정':<12} {'청크수':<8} {'크기':<6} {'겹침':<6} {'특징'}")
print("-" * 80)
for row in comparison_data:
    print(f"{row[0]:<12} {row[1]:<8} {row[2]:<6} {row[3]:<6} {row[4]}")

# 선택적: 파일로 저장
save_option = input("\n💾 결과를 파일로 저장하시겠습니까? (y/n): ")
if save_option.lower() == "y":
    save_chunks_to_file(chunks_1, "chunks_basic.txt", "기본 설정")
    save_chunks_to_file(chunks_2, "chunks_small.txt", "작은 청크")
    save_chunks_to_file(chunks_3, "chunks_korean.txt", "한국어 특화")
    save_chunks_to_file(chunks_4, "chunks_overlap.txt", "큰 겹침")

🔧 출력 제한 해제 완료!
🚀 RecursiveCharacterTextSplitter 파라미터별 비교

📊 기본 설정 (chunk_size=1000)
📈 통계:
  • 총 청크 수: 46
  • 평균 길이: 650.0자
  • 최소 길이: 3자
  • 최대 길이: 991자

📄 청크 1/46 (39자):
   전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동...

📄 청크 46/46 (707자):
   -30-
 ④ AI 일상화 기반을 선제적으로 조성하겠습니다추진과제추진시기소관부처▶ AI 문해력 제고AI 리터러시 함양 교수학습자료 개발 및 인정과목 개설’23교육부과기정통부디지털 튜터 배치 및 디지털 문제해결센터 운영’23교육부SW 미래채움센터 운영(’23년 13개 지역) ’23과기정통부AI 선도학교 운영 및 신규 AI 교육 콘텐츠 제공’23교육부과...

📄 청크 24/46 (908자):
   -13-
 <1-3> (보육･교육) 아동·청소년 성장환경 개선◇ 보육자 중심의 아이돌봄 플랫폼을 운영하여 양육 부담을 해소하고, AI 기반의 급식 관리 시스템을 통해 균형 있고 안전한 급식 제공◇ AI 디지털교과서를 개발･도입하여 학생 수준별 개별화된 학습지원□ 어린이･보호자의 안전･편의성을 향상하는 보육환경 구축○(돌봄 지원) 이용자 요구사항(돌봄방식,일...


📊 작은 청크 (chunk_size=300)
📈 통계:
  • 총 청크 수: 167
  • 평균 길이: 189.4자
  • 최소 길이: 1자
  • 최대 길이: 300자

📄 청크 1/167 (39자):
   전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동...

📄 청크 84/167 (300자):
   -13-
 <1-3> (보육･교육) 아동·청소년 성장환경 개선◇ 보육자 중심의 아이돌봄 플랫폼을 운영하여 양육 부담을 해소하고, AI 기반의 급식 관리 시스템을 통해 균형 있고 안전한 급식 제공◇ AI 디지털교과서를 개발･도입하여 학생 수준별 개별화된 학습

In [20]:
# 2. RecursiveCharacterTextSplitter
print("\n--- 2. RecursiveCharacterTextSplitter 실행 중... ---")
# recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=0, chunk_overlap=0, separators=[], keep_separator=False)
# recursive_chunks = recursive_splitter.split_documents(documents)
# for i, chunk in enumerate(recursive_chunks[:5]):
#     print(f"청크 예시:[{i}]")
#     print(f"CharacterTextSplitter로 분할된 청크 수: {len(recursive_chunks)}")
#     print("Content:")
#     print(chunk.page_content)
#     print("-" * 50)

recursive_splitter_1 = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", ". ", " ", ""], keep_separator=False
)
chunks_1 = recursive_splitter_1.split_documents(documents)
smart_display_chunks(chunks_1, "기본 설정 (chunk_size=1000)")


--- 2. RecursiveCharacterTextSplitter 실행 중... ---

📊 기본 설정 (chunk_size=1000)
📈 통계:
  • 총 청크 수: 46
  • 평균 길이: 650.0자
  • 최소 길이: 3자
  • 최대 길이: 991자

📄 청크 1/46 (39자):
   전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동...

📄 청크 46/46 (707자):
   -30-
 ④ AI 일상화 기반을 선제적으로 조성하겠습니다추진과제추진시기소관부처▶ AI 문해력 제고AI 리터러시 함양 교수학습자료 개발 및 인정과목 개설’23교육부과기정통부디지털 튜터 배치 및 디지털 문제해결센터 운영’23교육부SW 미래채움센터 운영(’23년 13개 지역) ’23과기정통부AI 선도학교 운영 및 신규 AI 교육 콘텐츠 제공’23교육부과...

📄 청크 24/46 (908자):
   -13-
 <1-3> (보육･교육) 아동·청소년 성장환경 개선◇ 보육자 중심의 아이돌봄 플랫폼을 운영하여 양육 부담을 해소하고, AI 기반의 급식 관리 시스템을 통해 균형 있고 안전한 급식 제공◇ AI 디지털교과서를 개발･도입하여 학생 수준별 개별화된 학습지원□ 어린이･보호자의 안전･편의성을 향상하는 보육환경 구축○(돌봄 지원) 이용자 요구사항(돌봄방식,일...



### SemanticChunker
- 임베딩(Embedding) 기반의 의미 단위(semantic) 분할
- 문장 간의 의미적 유사도(semantic similarity)를 계산해,
    → 의미 변화가 큰 경계에서 청크를 분할
- 구분자/문자수와 무관하게 내용 전개상의 의미 단위를 잘라냄
    → 자연어 처리, 요약 등에서 품질이 가장 좋음
- 임베딩 모델(예: OpenAI, HuggingFace 등)이 필요

#### 파라미터
- `embeddings` : 유사도 계산에 사용할 임베딩 모델	
  
- `breakpoint_threshold_type` : 경계 판단 방식 ('percentile', 'standard_deviation' 등)
    - `"percentile"` : 유사도 변화량이 상위 n% (예: 95%) 이상일 때
        - 예) 변화량 분포에서 상위 5% 지점  
        `breakpoint_threshold_type="percentile",`  
        `breakpoint_threshold_amount=95,`  
          
    - `"standard_deviation"` : 평균+표준편차 이상의 변화가 나올 때 (예: 변화량 평균+1σ 넘는 경우)
        - 예) 평균 + 1 표준편차 이상 변화에서 분할   
        `breakpoint_threshold_type="standard_deviation",`    
        `breakpoint_threshold_amount=1,`    
          
    - `"interquartile"` : 3사분위(Q3)+1.5×IQR 이상 변화일 때
      
    - `"absolute"` : 고정 임계값 지정 
        - 예) 0.7 이상 변화면 분할   
        `breakpoint_threshold_type = "absolute,"`  
        `breakpoint_threshold_amount = 0.7,`  
          
- `breakpoint_threshold_amount` : 임계값(퍼센트 또는 표준편차 등)
- `minimum_chunk_size` : 최소 청크 길이(문장/문자/토큰 단위 등, 구현체마다 다름)

In [21]:
# 3. SemanticChunker
# SemanticChunker는 임베딩 모델을 사용하여 문장 간의 의미적 거리를 계산합니다.
# breakpoint_threshold_type을 'percentile'로 설정하면, 문장 간 유사도 변화가 상위 n%에 해당할 때 분할합니다.
print("\n--- 3. SemanticChunker 실행 중... ---")
semantic_splitter = SemanticChunker(
    embeddings=embeddings, breakpoint_threshold_type="percentile", breakpoint_threshold_amount=95
)
semantic_chunks = semantic_splitter.split_documents(documents)
for i, chunk in enumerate(semantic_chunks[:5]):
    print(f"청크 예시:[{i}]")
    print(f"CharacterTextSplitter로 분할된 청크 수: {len(semantic_chunks)}")
    print("Content:")
    print(chunk.page_content[:500] + "..." if len(chunk.page_content) > 500 else chunk.page_content)
    print("-" * 50)


--- 3. SemanticChunker 실행 중... ---
청크 예시:[0]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
전국민 AI 일상화 실행계획
,
2023. 9.
--------------------------------------------------
청크 예시:[1]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
관 계 부 처  합 동
--------------------------------------------------
청크 예시:[2]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ.
--------------------------------------------------
청크 예시:[3]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
추진과제 10  1. AI로 국민 일상을 풍요롭게 하겠습니다10  2. AI 내재화로 산업･일터를 혁신하겠습니다15  3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20  4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------------------------------------
청크 예시:[4]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
요  약
--------------------------------------------------


### To Do
파라미터를 조절해 결과를 비교해보세요.

In [22]:
# 3. SemanticChunker
print("\n--- 3. SemanticChunker 실행 중... ---")
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",  # 또는 "standard_deviation", "interquartile"
    breakpoint_threshold_amount=95,
)
semantic_chunks = semantic_splitter.split_documents(documents)
for i, chunk in enumerate(semantic_chunks[:5]):
    print(f"청크 예시:[{i}]")
    print(f"CharacterTextSplitter로 분할된 청크 수: {len(semantic_chunks)}")
    print("Content:")
    print(chunk.page_content[:500] + "..." if len(chunk.page_content) > 500 else chunk.page_content)
    print("-" * 50)


--- 3. SemanticChunker 실행 중... ---
청크 예시:[0]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
전국민 AI 일상화 실행계획
,
2023. 9.
--------------------------------------------------
청크 예시:[1]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
관 계 부 처  합 동
--------------------------------------------------
청크 예시:[2]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ.
--------------------------------------------------
청크 예시:[3]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
추진과제 10  1. AI로 국민 일상을 풍요롭게 하겠습니다10  2. AI 내재화로 산업･일터를 혁신하겠습니다15  3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20  4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------------------------------------
청크 예시:[4]
CharacterTextSplitter로 분할된 청크 수: 44
Content:
요  약
--------------------------------------------------


### 각 청킹 결과로 VectorStore 생성 및 검색(Retrieval) 비교

이제 각기 다른 방식으로 분할된 청크들을 임베딩하여 별도의 VectorStore에 저장합니다.  
그리고 동일한 질문에 대해 각 VectorStore가 어떤 결과를 반환하는지 비교해 보겠습니다.  

In [23]:
def create_vector_store(chunks, embeddings):
    """주어진 청크와 임베딩 모델로 FAISS VectorStore를 생성하고 retriever를 반환합니다."""
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store.as_retriever()

In [24]:
# 각 청킹 전략별로 retriever 생성
retrievers = {
    "Character": create_vector_store(char_chunks, embeddings),
    "Recursive": create_vector_store(recursive_chunks, embeddings),
    "Semantic": create_vector_store(semantic_chunks, embeddings),
}

In [25]:
# 비교를 위한 질문
query = "AI 일상화 추진전략의 주요 내용은?"

print("\n\n" + "=" * 50)
print(f"질문: {query}")
print("=" * 50 + "\n")



질문: AI 일상화 추진전략의 주요 내용은?



In [26]:
# 각 retriever의 검색 결과 비교
for name, retriever in retrievers.items():
    print(f"--- {name} Splitter의 검색 결과 ---")
    retrieved_docs = retriever.invoke(query)
    # 검색된 문서의 내용을 출력
    for i, doc in enumerate(retrieved_docs):
        print(f"  [문서 {i + 1}]")
        print(doc.page_content)
        print("-" * 20)
    print("\n")

--- Character Splitter의 검색 결과 ---
  [문서 1]
순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ. 추진과제 10  1. AI로 국민 일상을 풍요롭게 하겠습니다10  2. AI 내재화로 산업･일터를 혁신하겠습니다15  3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다20  4. AI 일상화 기반을 선제적으로 조성하겠습니다24 Ⅵ. 추진체계 및 계획 27
--------------------
  [문서 2]
추진체계 및 계획□ 추진체계 및 향후계획ㅇ ’정보통신전략위원회‘를 통해 동 실행계획의 추진실적 점검 ㅇ 각 부처 소관 영역의 AI일상화 신규과제 발굴,예산 반영 협의ㅇ 개발･
실증 및 현장적용 단계에서 효과가 검증된 AI제품･
서비스에대해서는 민관 협업을 통해 대규모 프로젝트로 추진□ 과제 추진계획 ① AI로 국민 일상을 풍요롭게 하겠습니다추진과제추진시기소관부처▶ (복지) 사회적 약자 돌봄･배려독거노인 등 어르신 대상 건강관리･돌봄 서비스 제공‘23복지부보육원 보호아동 대상 학습, 정서안정, 문해력 향상 지원  ‘25(잠정)과기정통부청각장애인 대상 AI 보조기기 지원‘23과 기 정 통･고 용 부장애인 재활운동 및 운동코칭 AI 기기･솔루션 실증･제공‘23과기정통부복지부, 문체부AI 기반 복지 위기가구 포착 및 복지수요 확인‘24복지부AI 초기상담 서비스 체계 구축‘24복지부▶ (건강) 의료･보건 서비스 품질 제고중증질환 및 소아희귀질환 진단･관리 AI 개발‘23과기정통부자폐성 장애 치료･관리를 위한 디지털치료기기 개발‘23과기정통부복지부의료기관 대상 클라우드 병원정보시스템, 질환진단 AI 도입 지원  ‘23과기정통부감염병 확산 예측 AI 모델 및 대응 시나리오 개발‘23질병청AI 일일모기발생감시장비(AI-DMS) 운영‘24질병청▶ (보육･교육) 아동･청소년 성장 환경 개선아이돌봄플랫폼 구축･운영 ‘23여가부맞춤형 식단,

## 비교용 대표 질문 예시
1. 전략/정책 요약 파악용

  - AI 일상화를 위한 핵심 전략은 무엇인가요?”

  - 복지 분야에서 AI는 어떤 방식으로 활용되나요?”

  - AI를 활용한 행정 혁신 방안은 무엇인가요?”

2. 세부 내용 탐색용

  - AI 기반 복지 서비스의 도입 시기는 언제인가요?”

  - AI 디지털 교과서는 언제 도입될 예정인가요?”

  - AI 정수장 구축은 어떤 기관에서 검토하고 있나요?”

3. 추론/통합형

  - AI 기술은 어떤 방식으로 국민 생활 전반에 영향을 미치나요?”

  - 교육과 보건 분야의 AI 정책 간의 공통된 접근 방식은 무엇인가요?”

  - 각 부처는 AI 일상화를 위해 어떤 책임을 갖고 있나요?”

4. 오류 유도형 (Chunking 성능 한계점 테스트)

  - AI기반 급식관리 시스템을 도입한 부처는 어디인가요?”

  - AI기반 감염병 대응 모델 개발은 어느 시기에 시행되나요?”

  - 정보통신전략위원회는 어떤 역할을 하나요?”

## TIP
- 짧은 질문은 Character나 Recursive Split이 잘 대응할 수 있고,
- 의미 중심의 질문은 Semantic이 유리합니다.